In [1]:
import psycopg2
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from numpy import random

from sklearn.preprocessing import StandardScaler

import os
os.chdir('/home/rafalb/work/molecules/moleculesStatic')

### Database connection

In [2]:
connection = psycopg2.connect(user = "chembl",
                              password = "chembl",
                              host = "127.0.0.1",
                              port = "5432",
                              database = "chembl_25")
cursor = connection.cursor()

### Types of activities

In [3]:
cursor.execute('select min(AC.standard_type) AS TYPE, count(AC.standard_type) AS COUNT from ACTIVITIES AC GROUP BY standard_type')
activities = pd.DataFrame(cursor.fetchall(), columns = ['type', 'count'])

In [4]:
activities[activities['count'] > 100000]

,type,count
110,AC50,156790
145,Activity,839702
1694,EC50,394406
2115,GI50,2579793
2401,IC50,2049897
2561,Inhibition,959104
2868,Ki,687703
3878,MIC,544741
4762,Potency,4473542
5798,Tissue Severity Score,128999


### Main query

In [5]:
columns = ['molregno', 'canonical_smiles', 'activity_id', 
           'standard_value', 'standard_units', 'standard_flag', 'standard_type', 'activity_comment',
           'alogp', 'hba', 'hbd', 'psa', 'rtb', 'ro3_pass', 'num_ros_violations', 'molecular_species', 'full_mwt', 'aromatic_rings', 'heavy_atoms', 'qed_weighted']

cursor.execute("select CS.molregno, \
               CS.canonical_smiles, \
               AC.activity_id, \
               AC.standard_value, \
               AC.standard_units, \
               AC.standard_flag, \
               AC.standard_type, \
               AC.activity_comment, \
               CP.ALOGP, \
               CP.HBA, \
               CP.HBD, \
               CP.PSA, \
               CP.RTB, \
               CP.RO3_PASS, \
               CP.NUM_RO5_VIOLATIONS, \
               CP.MOLECULAR_SPECIES, \
               CP.FULL_MWT, \
               CP.AROMATIC_RINGS, \
               CP.HEAVY_ATOMS, \
               CP.QED_WEIGHTED \
               from COMPOUND_STRUCTURES CS \
               inner join ACTIVITIES AC on CS.molregno = AC.molregno \
               inner join COMPOUND_PROPERTIES CP on CS.molregno = CP.MOLREGNO \
               and (AC.standard_type = 'IC50' or AC.standard_type = 'GI50' or AC.standard_type = 'Potency') \
               and (AC.standard_value IS NOT NULL)")
molData = pd.DataFrame(cursor.fetchall(), columns = columns)
connection.close()

In [6]:
molData.shape

(8821755, 20)

In [7]:
molData.head(10).iloc[1,:]

molregno                                     61021
canonical_smiles      Brc1ccc(NC(=S)NCN2CCOCC2)nc1
activity_id                                  32098
standard_value                              3800.0
standard_units                                  nM
standard_flag                                    1
standard_type                                 IC50
activity_comment                              None
alogp                                         1.42
hba                                              4
hbd                                              2
psa                                          49.42
rtb                                              3
ro3_pass                                         N
num_ros_violations                               0
molecular_species                          NEUTRAL
full_mwt                                    331.24
aromatic_rings                                   1
heavy_atoms                                     18
qed_weighted                   

### Data conversion

In [8]:
floatDescriptors = ['standard_value', 'alogp', 'psa', 'full_mwt', 'qed_weighted']
for moldesc in floatDescriptors:
    molData[moldesc] = molData[moldesc].astype(float)

intDescriptors = ['molregno']#, 'hba', 'hbd', 'rtb', 'aromatic_rings', 'heavy_atoms']
for moldesc in intDescriptors:
    molData[moldesc] = molData[moldesc].astype(int)

### What units do we have?

In [9]:
# mess in units
molData.groupby('standard_units').agg('count')

,molregno,canonical_smiles,activity_id,standard_value,standard_flag,standard_type,activity_comment,alogp,hba,hbd,psa,rtb,ro3_pass,num_ros_violations,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted
standard_units,,,,,,,,,,,,,,,,,,,
%,384,384,384,384,384,384,0,241,241,241,241,241,241,241,224,384,241,241,241
% conc,70,70,70,70,70,70,0,70,70,70,70,70,70,70,70,70,70,70,70
/uM,25,25,25,25,25,25,0,25,25,25,25,25,25,25,25,25,25,25,25
/uM/s,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1
10'-10 uM,4,4,4,4,4,4,0,4,4,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ug well-1,12,12,12,12,12,12,0,12,12,12,12,12,12,12,12,12,12,12,12
ug.mL-1,72897,72897,72897,72897,72897,72897,34,70270,70270,70270,70270,70270,70270,70270,63184,72897,70270,70270,70270
ug/g,18,18,18,18,18,18,0,18,18,18,18,18,18,18,5,18,18,18,18


In [10]:
# take only the entries expressed in nM
molData = molData[molData['standard_units']=='nM'].reset_index()
molData.groupby('standard_units').agg('count')

,index,molregno,canonical_smiles,activity_id,standard_value,standard_flag,standard_type,activity_comment,alogp,hba,hbd,psa,rtb,ro3_pass,num_ros_violations,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted
standard_units,,,,,,,,,,,,,,,,,,,,
nM,8744913,8744913,8744913,8744913,8744913,8744913,8744913,7100433,8629661,8629661,8629661,8629661,8629661,8629661,8629661,8186771,8744913,8629661,8629661,8629661


In [11]:
molData.head(5)

,index,molregno,canonical_smiles,activity_id,standard_value,standard_units,standard_flag,standard_type,activity_comment,alogp,...,hbd,psa,rtb,ro3_pass,num_ros_violations,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted
0,0,61021,Brc1ccc(NC(=S)NCN2CCOCC2)nc1,32097,10000.0,nM,1,IC50,None,1.42,...,2.0,49.42,3.0,N,0.0,NEUTRAL,331.24,1.0,18.0,0.82
1,1,61021,Brc1ccc(NC(=S)NCN2CCOCC2)nc1,32098,3800.0,nM,1,IC50,None,1.42,...,2.0,49.42,3.0,N,0.0,NEUTRAL,331.24,1.0,18.0,0.82
2,2,61021,Brc1ccc(NC(=S)NCN2CCOCC2)nc1,32099,1000.0,nM,1,IC50,None,1.42,...,2.0,49.42,3.0,N,0.0,NEUTRAL,331.24,1.0,18.0,0.82
3,3,27307,CC1=CN([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)C...,32102,4.0,nM,1,IC50,None,-0.20,...,2.0,133.08,3.0,N,0.0,NEUTRAL,267.25,1.0,19.0,0.45
4,4,27307,CC1=CN([C@H]2C[C@H](N=[N+]=[N-])[C@@H](CO)O2)C...,32103,150.0,nM,1,IC50,None,-0.20,...,2.0,133.08,3.0,N,0.0,NEUTRAL,267.25,1.0,19.0,0.45


### Data aggregation

In [12]:
aggFunctions = {
                'molregno': ['min', 'count'], 'canonical_smiles': 'min',
                'standard_value': ['min', 'max'],
                'standard_type': 'min',
                'alogp': ['min', 'max'],
                'hba': ['min', 'max'],
                'hbd': ['min', 'max'],
                'psa': ['min', 'max'],
                'rtb': ['min', 'max'],
                'ro3_pass': 'min',
                'num_ros_violations': 'min',
                'molecular_species': 'min',
                'full_mwt': ['min', 'max'],
                'aromatic_rings': 'min',
                'heavy_atoms': 'min',
                'qed_weighted': ['min', 'max']
                }
grouped = molData.groupby('molregno')
molDataGrouped = grouped.agg(aggFunctions).reset_index()

In [13]:
molDataGrouped.head(5)

molregno                                             canonical_smiles  \
           min count                                                min   
0       10  10     2  C1CCCCCNc2cc[n+](Cc3cccc(c3)c4cccc(C[n+]5ccc(N...   
1       23  23    64                Br\C=C\1/CCC(C(=O)O1)c2cccc3ccccc23   
2       24  24     1                 I\C=C\1/CCC(C(=O)O1)c2cccc3ccccc23   
3       25  25     1                 O=C1O\C(=C\C#C)\CCC1c2cccc3ccccc23   
4       26  26     1                 I\C=C/1\CCC(C(=O)O1)c2cccc3ccccc23   

  standard_value          standard_type alogp        hba  ...  rtb ro3_pass  \
             min      max           min   min   max  min  ...  max      min   
0          110.0    110.0          IC50  9.29  9.29  2.0  ...  0.0        N   
1            3.2  73078.0          IC50  4.50  4.50  2.0  ...  1.0        N   
2           30.0     30.0          IC50  4.54  4.54  2.0  ...  1.0        N   
3           95.0     95.0          IC50  3.78  3.78  2.0  ...  1.0        N   
4          190.0    190.0          IC50  4.54  4.54  2.0  ...  1.0        N   

  num_ros_violations molecular_species full_mwt         aromatic_rings  \
                 min               min      min     max            min   
0                2.0           NEUTRAL   606.86  606.86            6.0   
1                0.0               NaN   317.18  317.18            2.0   
2                0.0               NaN   364.18  364.18            2.0   
3                0.0               NaN   262.31  262.31            2.0   
4                0.0               NaN   364.18  364.18            2.0   

  heavy_atoms qed_weighted        
          min          min   max  
0        46.0         0.17  0.17  
1        19.0         0.72  0.72  
2        19.0         0.55  0.55  
3        20.0         0.58  0.58  
4        19.0         0.55  0.55  

[5 rows x 26 columns]

### Activity type distribution

In [14]:
# THE GI50/IC50 distribution
molDataGrouped['standard_type'].groupby('min').agg({'min': ['count']})

,min
,count
min,
GI50,59431
IC50,695589
Potency,374725


### Pickling of the data

In [ ]:
import pickle
pcklFile = 'molDataGrouped.pckl'

In [ ]:
with open(pcklFile, 'wb') as file:
    pickle.dump(molDataGrouped, file)

In [ ]:
with open(pcklFile, 'rb') as file:
    molDataGrouped = pickle.load(file)

In [ ]:
molDataGrouped.columns

In [ ]:
molDataGrouped.loc[:, ['canonical_smiles', 'min']].values

In [ ]:
molDataGrouped.head(10)

### Smiles analysis

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Draw

In [ ]:
def provideMoleculeStatistics(smiles):
    #print(smiles)
    mol = Chem.MolFromSmiles(smiles)
    
    newSmiles = Chem.MolToSmiles(mol, canonical = True, isomericSmiles = False)
    negativeCharged = sum([ item.GetFormalCharge() if item.GetFormalCharge() < 0 else 0 for item in mol.GetAtoms() ])
    positiveCharged = sum([ item.GetFormalCharge() if item.GetFormalCharge() > 0 else 0 for item in mol.GetAtoms() ])
    #anyCharged = any([item1 or item2 for item1, item2 in zip(negativelyCharged, positivelyCharged)])
    
    elementsList = list(set([atom.GetSymbol() for atom in mol.GetAtoms()]))
    numberOfRings = mol.GetRingInfo().NumRings()
    
    return(newSmiles, negativeCharged, positiveCharged, elementsList, numberOfRings)


In [ ]:
import codecs
encodeToUTF8 = False
def canonicalizeSmilesAndProvideDescriptor(smiles):
    #rdkitMol = Chem.MolFromSmiles(molecule)
    #Chem.SanitizeMol(rdkitMol)
    try:
        #newSmiles = Chem.MolToSmiles(rdkitMol, canonical = True, isomericSmiles = False)
        newSmiles, negativeCharged, positiveCharged, elementsList, numberOfRings = provideMoleculeStatistics(smiles)
        #smilesDescription = checkVariousSmilesProperties(newSmiles)
        #elementsSet = provideElementsList(newSmiles)        
    except:
        newSmiles, negativeCharged, positiveCharged, elementsList, numberOfRings = (None, None, None, None, None)
        # There was a trouble in catching the ArgumentError exception (originatefd most likely in Boost.Python 
        # therefore any exceptio s caught here)
        print('Exception!!! :', smiles)
        
    if (encodeToUTF8):
        return((codecs.encode(newSmiles, 'utf-8'), negativeCharged, positiveCharged, elementsList, numberOfRings))
    else:
        return((newSmiles, negativeCharged, positiveCharged, elementsList, numberOfRings))

In [ ]:
print(pd.__version__)

In [ ]:
def printRow(row):
    print('in')
    print(row)
    mol = Chem.MolFromSmiles(row)

In [ ]:

molDataGrouped.loc[:2, ('canonical_smiles', 'min')].apply(printRow)
 #.apply(canonicalizeSmilesAndProvideDescriptor)

In [ ]:
len(molDataGrouped)

In [ ]:
sourceColumn = ('canonical_smiles', 'min')
nTotal = len(molDataGrouped)
nStart = 0
nSize = 10000
nBatch = np.ceil((nTotal - nStart)/nSize).astype(int)
for iii in range(nBatch):
    iBeg = nStart + iii * nSize
    if (iii == nBatch - 1):
        iEnd = nTotal
    else:
        iEnd = nStart + (iii + 1) * nSize
    print(iii)
    result = molDataGrouped.loc[iBeg:iEnd, sourceColumn].apply(canonicalizeSmilesAndProvideDescriptor)
    molDataGrouped.loc[iBeg:iEnd, 'canonicalSmiles'] = [item[0] for item in result]
    molDataGrouped.loc[iBeg:iEnd, 'negativeCharged'] = [item[1] for item in result]
    molDataGrouped.loc[iBeg:iEnd, 'positiveCharged'] = [item[2] for item in result]
    molDataGrouped.loc[iBeg:iEnd, 'elementsSet'] = [item[3] for item in result]
    molDataGrouped.loc[iBeg:iEnd, 'numberOfRings'] = [item[4] for item in result]

In [ ]:
molDataGrouped.head(4)

In [ ]:
molDataGrouped[molDataGrouped['elementsSet'] == None]

In [ ]:
with open('molDataGroupedDesc.pckl', 'wb') as file:
    pickle.dump(molDataGrouped, file)

In [ ]:
with open('molDataGroupedDesc.pckl', 'rb') as file:
    molDataGrouped = pickle.load(file)

In [ ]:
molDataGrouped = molDataGrouped[~molDataGrouped['elementsSet'].isnull()]

In [ ]:
organicChemistryList = ['B', 'C', 'N', 'O', 'P', 'S', 'F', 'Cl', 'Br', 'I']
organicChemistrySet = set(organicChemistryList)

In [ ]:
testSet = set(['N', 'C', 'Cl'])
testSet < organicChemistrySet

In [ ]:
molDataGrouped['organicChemistryElementsOnly'] = molDataGrouped['elementsSet'].apply(lambda x: set(x) < organicChemistrySet)

In [ ]:
molDataGrouped.groupby('organicChemistryElementsOnly').count()

In [ ]:
molDataGrouped['canonicalSmilesLength'] = molDataGrouped['canonicalSmiles'].apply(lambda x: len(x))

In [ ]:
plt.hist(molDataGrouped['canonicalSmilesLength'], bins = 50)

In [ ]:
limitSmilesLength = 100
molDataGroupedChosen = molDataGrouped[(molDataGrouped['canonicalSmilesLength'] < limitSmilesLength) & \
                      (molDataGrouped['negativeCharged'] == 0) & \
                      (molDataGrouped['positiveCharged'] == 0) & \
                      (molDataGrouped['numberOfRings'] <= 5) & \
                      (molDataGrouped['organicChemistryElementsOnly'])].reset_index()

In [ ]:
replacementDict = {'Br': 'G', 'Cl': 'U', '[nH]': 'W'}
molDataGroupedChosen['encodedSmiles'] = molDataGroupedChosen['canonicalSmiles'].replace(replacementDict, regex=True)

In [ ]:
molDataGroupedChosen.head(5)

In [ ]:
molDataGroupedChosen.shape

In [ ]:
# molDataGroupedFinal_100.pckl: smiles length < 100
with open('molDataGroupedFinal_100.pckl', 'wb') as file:
    pickle.dump(molDataGroupedChosen, file)

In [ ]:
with open('molDataGroupedFinal.pckl', 'rb') as file:
    molDataGroupedChosen = pickle.load(file)


In [ ]:
molDataGroupedChosen.head()

In [ ]:
nSmilesCodes = 50000
mask = random.randint(0, molDataGroupedChosen.shape[0], size=nSmilesCodes)

In [ ]:
staticFeatures = pd.DataFrame()
toBeAveraged = ['standard_value', 'alogp', 'hba', 'hbd', 'psa', 'rtb', 'full_mwt', 'qed_weighted']
for quantity in toBeAveraged:
    staticFeatures.loc[:, quantity] = (molDataGroupedChosen.loc[mask, (quantity, 'min')] + molDataGroupedChosen.loc[mask, (quantity, 'max')])/2
    staticFeatures.loc[:, quantity].astype(float)
toBeTaken = ['aromatic_rings', 'heavy_atoms']
for quantity in toBeTaken:
    staticFeatures.loc[:, quantity] = molDataGroupedChosen.loc[mask, (quantity, 'min')]
    staticFeatures.loc[:, quantity].astype(float)
staticFeatures.loc[:, 'number_of_rings'] = molDataGroupedChosen.loc[mask, 'numberOfRings'].astype(float)
staticFeatures.loc[:, 'number_of_rings'] = staticFeatures.loc[:, 'number_of_rings'].astype(float)
print(staticFeatures.head(2))
#staticFeatures = staticFeatures.values

In [ ]:
staticFeatures['full_mwt'] = staticFeatures.full_mwt.astype(float)
staticFeatures['qed_weighted'] = staticFeatures.qed_weighted.astype(float)
staticFeatures['aromatic_rings'] = staticFeatures.qed_weighted.astype(float)


In [ ]:
thres = 100000
staticFeatures[staticFeatures['standard_value'] < thres].shape[0] / staticFeatures['standard_value'].shape[0]

In [ ]:
staticFeatures = staticFeatures[staticFeatures['standard_value'] < thres]

In [ ]:
staticFeatures.shape

In [ ]:
allDescriptors = ['standard_value', 'alogp', 'hba', 'hbd', 'psa', 'rtb', 'full_mwt', 'qed_weighted', 'aromatic_rings', 'heavy_atoms', 'number_of_rings']
#allDescriptors = ['standard_value']
#quantity = 'alogp'
plotIdx = 1
nRows = np.ceil(len(allDescriptors) / 2)
fig = plt.figure(figsize=(16, 16)) 
for quantity in allDescriptors:
    print(quantity)
    plt.subplot(nRows, 2, plotIdx)
    plt.hist(staticFeatures[~staticFeatures[quantity].isnull()][quantity], bins = 10)
    plt.title(quantity)
    plotIdx += 1


## Dynamic features

In [ ]:
smilesCodes = molDataGroupedChosen.loc[staticFeatures.index, 'encodedSmiles']
smilesCodes

In [ ]:
maxlen = -1
for code in smilesCodes:
    if len(code) > maxlen:
        maxlen = len(code)
maxlen

In [ ]:
minlen = 1e6
for code in smilesCodes:
    if len(code) < minlen:
        minlen = len(code)
minlen

In [ ]:
def pad_smile(string, max_len, padding='right'):
    if len(string) <= max_len:
        if padding == 'right':
            return string + " " * (max_len - len(string))
        elif padding == 'left':
            return " " * (max_len - len(string)) + string
        elif padding == 'none':
            return string


In [ ]:
smilesCodes = smilesCodes.apply(lambda x: pad_smile(x, max_len=maxlen, padding='right'))

In [ ]:
chars = sorted(list(set(smilesCodes.str.cat(sep=''))))
print('total chars:', len(chars))
char2indices = dict((c, i) for i, c in enumerate(chars))
indices2char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
dynamicFeatures = np.zeros((len(smilesCodes), maxlen, len(chars)), dtype=np.float)
dynamicFeatures.shape

In [ ]:
for codeidx, code in enumerate(smilesCodes):
    for charidx, char in enumerate(code):
        dynamicFeatures[codeidx, charidx, char2indices[char]] = 1

In [ ]:
sums = []
for idx in range(dynamicFeatures.shape[0]):
    sums.append(np.sum(dynamicFeatures[idx, :, :]))
plt.hist(sums)

In [ ]:
staticChosen = ['alogp', 'full_mwt']
scaler = StandardScaler()
scaler.fit(staticFeatures[staticChosen])

In [ ]:
staticFeaturesStandard = scaler.transform(staticFeatures[staticChosen])

## Autoencoder architecture

In [ ]:
from keras.layers import LSTM, TimeDistributed, concatenate, Input, Dense, RepeatVector, Lambda
from keras.models import Model
from keras.activations import relu, sigmoid, tanh
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, RMSprop
import keras.backend as K
from keras.utils import plot_model
from keras import losses
import numpy.random as rnd

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [ ]:
def prepare_model(static, dynamic, k, window, charsetLen, lr, lossFunction, showArch):
    input_dynamic = Input(shape=(window, charsetLen), name="input_dynamic")
    input_static = Input(shape=(static,), name="input_static")
    latent = Dense(k[0], activation=relu)(input_static)
    dense_h = Dense(k[0])(latent)
    dense_c = Dense(k[0])(latent)
    lstm_layer, state_h, state_c = LSTM(k[0], return_sequences=True, return_state=True)(input_dynamic,
                                                                                        initial_state=[dense_h,
                                                                                                       dense_c])

    for x in k[1:-1]:
        concat_h = concatenate([dense_h, state_h])
        dense_h = Dense(x)(concat_h)
        concat_c = concatenate([dense_c, state_c])
        dense_c = Dense(x)(concat_c)
        lstm_layer, state_h, state_c = LSTM(x, return_sequences=True, return_state=True)(lstm_layer,
                                                                                         initial_state=[dense_h,
                                                                                                        dense_c])
    x = k[-1]
    concat_h = concatenate([dense_h, state_h])
    dense_h = Dense(x)(concat_h)
    concat_c = concatenate([dense_c, state_c])
    dense_c = Dense(x)(concat_c)
    lstm_layer, state_h, state_c = LSTM(x, return_state=True)(lstm_layer, initial_state=[dense_h, dense_c])
    concat = concatenate([lstm_layer, latent])

    # autoencoder
    z_mean = Dense(x, name='z_mean')(concat)
    z_log_var = Dense(x, name='z_log_var')(concat)
    
    
    z = Lambda(sampling, output_shape=(x,), name='z')([z_mean, z_log_var])
    
    
    
    state_h = Dense(k[-2], activation=relu)(z)
    dense_h = Dense(k[-2], activation=relu)(z)
    state_c = Dense(k[-2], activation=relu)(z)
    dense_c = Dense(k[-2], activation=relu)(z)
    lstm_layer = RepeatVector(window)(z)

    for x in np.flip(k[:-1]):
        concat_h = concatenate([dense_h, state_h])
        dense_h = Dense(x)(concat_h)
        concat_c = concatenate([dense_c, state_c])
        dense_c = Dense(x)(concat_c)
        lstm_layer, state_h, state_c = LSTM(x, return_sequences=True, return_state=True)(lstm_layer,
                                                                                         initial_state=[dense_h,
                                                                                                        dense_c])

    #result_series = TimeDistributed(Dense(charsetLen))(lstm_layer)
    result_series = LSTM(charsetLen, return_sequences=True, activation='softmax')(lstm_layer)
    concat = concatenate([state_h, state_c])
    #result_sigmoid = Dense(static-3, activation=sigmoid)(concat)
    result_relu = Dense(static, activation=sigmoid)(concat)

    #model = Model(inputs=[input_dynamic, input_static], outputs=[result_series, result_sigmoid, result_relu])
    model = Model(inputs=[input_dynamic, input_static], outputs=[result_series, result_relu])
    optimizer = RMSprop(lr=lr)
    model.compile(optimizer=optimizer, loss=lossFunction, metrics=['binary_crossentropy', 'mean_absolute_error'])
    if (showArch):
        print(model.summary())
    return model

In [ ]:
def fit(staticFeatures, dynamicFeatures, model, step=1):
    #dynamic_data = np.empty((0, window, 1), np.float)
    #helper = []
    #for d in dynamic:
    #    new_data = rolling_window(d, window, step)
    #    helper.append(len(new_data))
    #    dynamic_data = np.append(dynamic_data, new_data, axis=0)
    #print(len(helper))
    #static_data = np.repeat(static, helper, axis=0)
    order = rnd.permutation(len(staticFeatures))

    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    bst_model_path = 'autoencoder.h5'
    checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True, monitor='val_loss')

    size = int(staticFeatures.shape[0] * 0.9)
    training_dynamic, training_static = dynamicFeatures[order[:size]], staticFeatures[order[:size]]
    testing_dynamic, testing_static = dynamicFeatures[order[size:]], staticFeatures[order[size:]]
    print(training_dynamic.shape, training_static.shape)
    print(testing_dynamic.shape, testing_static.shape)
    model.fit([training_dynamic, training_static], 
              [training_dynamic, training_static],
                   epochs=10,
                   batch_size=64,
                   callbacks=[early_stopping, checkpoint],
                   validation_data=([testing_dynamic, testing_static], 
                                    [testing_dynamic, testing_static]))

In [ ]:
lr = 0.001
model = prepare_model(staticFeaturesStandard.shape[1], 1, [64,64], dynamicFeatures.shape[1], dynamicFeatures.shape[2], lr, ['binary_crossentropy', 'mean_absolute_error'], True)

In [ ]:
fit(staticFeaturesStandard, dynamicFeatures, model)

In [ ]:
dynamicFeatures[0,:,:].reshape(-1, dynamicFeatures.shape[1], dynamicFeatures.shape[2])

In [ ]:
staticFeaturesStandard[0,:].reshape(-1, staticFeaturesStandard.shape[1]).shape

In [ ]:
prediction = model.predict([dynamicFeatures[0,:,:].reshape(-1, dynamicFeatures.shape[1], dynamicFeatures.shape[2]), staticFeaturesStandard[0,:].reshape(-1, staticFeaturesStandard.shape[1]) ])

In [ ]:
prediction[0]

In [ ]:
import SmilesEnumerator